# Document Analysis in Python
In this notebook we will cover:
- Reading document data into memory
- Creating bag of words features
- Creating smoothed tf-idf features

In [3]:
import requests
import json
from contextlib import closing

# get API key saved on hardrive
with open('../NYTimesAPI.txt') as f:
    api_key = f.read() # read in my private key (sorry, not in this repo ¯\_(ツ)_/¯ )
    
# make base URL and dictionary of get request key/values
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json"
payload = {'api-key': api_key, 'q':'McCain'} # key/values for get request (look up in api, there are lots)

# Perform the actual request
with closing(requests.get(url, params=payload)) as r:
    articles = r.json()
    
articles

{'status': 'OK',
 'copyright': 'Copyright (c) 2018 The New York Times Company. All Rights Reserved.',
 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2018/08/27/podcasts/the-daily/senator-john-mccain.html',
    'snippet': '“The Daily” looks at the senator’s complex legacy through his half-century of public life.',
    'blog': {},
    'source': 'The New York Times',
    'multimedia': [{'rank': 0,
      'subtype': 'xlarge',
      'caption': None,
      'credit': None,
      'type': 'image',
      'url': 'images/2018/08/27/podcasts/the-daily/27daily1/27daily1-articleLarge.jpg',
      'height': 400,
      'width': 600,
      'legacy': {'xlarge': 'images/2018/08/27/podcasts/the-daily/27daily1/27daily1-articleLarge.jpg',
       'xlargewidth': 600,
       'xlargeheight': 400},
      'subType': 'xlarge',
      'crop_name': 'articleLarge'},
     {'rank': 0,
      'subtype': 'wide',
      'caption': None,
      'credit': None,
      'type': 'image',
      'url': 'images/2018/08/27/po

In [ ]:
# OR we can load an example query like this
# run this block of code if you can't run anything else
import json 
with open('data/nytime.json') as f:
    articles = json.loads(f.read())
    
articles

In [4]:
# get the summary text from each article
#  lead_paragraph  is no snippet
summary_text = [x['snippet'] for x in articles['response']['docs']]
summary_text

['“The Daily” looks at the senator’s complex legacy through his half-century of public life.',
 'A reader says the president’s refusal to mention the ailing senator’s name while discussing a military bill named in his honor shows Mr. Trump’s immaturity and vindictiveness. ',
 'Readers offer praise for the senator and war hero who died on Saturday.',
 'A look at the formative times and turmoil that shaped a historic American figure, with Carl Hulse, The Times’s chief Washington correspondent.',
 'Reporters and editors at The New York Times had a backstage view of Mr. McCain’s decades in public life. Here are their reflections.',
 'The president avoided reading the name of the military spending bill — which would have meant mentioning the ailing senator and Trump critic — during his speech or the signing ceremony.',
 'The Senate becomes a lesser place without the sometimes angry, often funny, always larger-than-life presence of the Republican from Arizona.',
 'Mr. McCain won the Republic

# Converting document data into different representations
First lets go through and count the unique words in each opening sentence (that is what the NYTimes give us for free).
- http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer() # an object capable of counting words in a document!

# count_vect.fit(summary_text)
# count_vect.transform(summary_text)
bag_words = count_vect.fit_transform(summary_text)

In [6]:
print(bag_words.shape) # this is a sparse matrix
print('=========')
print(bag_words[0])

(10, 142)
  (0, 62)	1
  (0, 95)	1
  (0, 81)	1
  (0, 22)	1
  (0, 45)	1
  (0, 50)	1
  (0, 122)	1
  (0, 60)	1
  (0, 25)	1
  (0, 109)	1
  (0, 12)	1
  (0, 64)	1
  (0, 30)	1
  (0, 119)	2


# Self Test: ML01b.3: 
Do you expect the vocabulary from the articles above to be:
- A. Greater than 1M words
- B. Greater than 10,000 words
- C. Fewer than 10,000 words

In [7]:
print(len(count_vect.vocabulary_))
print(count_vect.vocabulary_)

142
{'the': 119, 'daily': 30, 'looks': 64, 'at': 12, 'senator': 109, 'complex': 25, 'legacy': 60, 'through': 122, 'his': 50, 'half': 45, 'century': 22, 'of': 81, 'public': 95, 'life': 62, 'reader': 97, 'says': 107, 'president': 93, 'refusal': 102, 'to': 124, 'mention': 70, 'ailing': 2, 'name': 74, 'while': 134, 'discussing': 33, 'military': 72, 'bill': 18, 'named': 75, 'in': 55, 'honor': 52, 'shows': 112, 'mr': 73, 'trump': 126, 'immaturity': 54, 'and': 7, 'vindictiveness': 130, 'readers': 98, 'offer': 82, 'praise': 91, 'for': 38, 'war': 131, 'hero': 49, 'who': 135, 'died': 32, 'on': 85, 'saturday': 106, 'look': 63, 'formative': 39, 'times': 123, 'turmoil': 127, 'that': 118, 'shaped': 111, 'historic': 51, 'american': 5, 'figure': 37, 'with': 136, 'carl': 21, 'hulse': 53, 'chief': 24, 'washington': 132, 'correspondent': 26, 'reporters': 103, 'editors': 35, 'new': 76, 'york': 141, 'had': 44, 'backstage': 14, 'view': 129, 'mccain': 68, 'decades': 31, 'here': 48, 'are': 9, 'their': 120, 'r

In [6]:
# we can still look at the data using an inverse transform
# but we lose the ordering of the words (after all its just a bag of wrods model)
count_vect.inverse_transform(bag_words[0])

[array(['anecdotes', 'lively', 'full', 'is', 'hoover', 'edgar', 'of',
        'files', 'private', 'the', 'and', 'alive', 'it', 'like', 'films',
        'made', 'who', 'cohen', 'larry', 'about', 'documentary', 'this'],
       dtype='<U13')]

In [8]:
# now let's create a pandas API out of this
import pandas as pd

pd.options.display.max_columns = 999
df = pd.DataFrame(data=bag_words.toarray(),columns=count_vect.get_feature_names())

In [8]:
df # display the full bag of words matrix  

,11,1990,2001,45th,about,acting,adam,alive,and,anecdotes,answers,archived,are,articles,at,attention,becoming,before,by,class,cocky,cohen,correspondent,could,decade,div,documentary,edgar,edito,editor,files,films,find,fixer,fn,for,foreign,full,he,hearings,himself,hoover,humbled,impeachment,in,is,it,joined,joyce,knows,larry,like,lines,lively,made,mistrials,more,new,news,noam,now,of,on,once,org,our,pardons,patricia,paying,president,private,questions,quick,recent,right,roger,sarah,sept,span,star,story,than,the,this,times,to,vcard,was,we,who,witness,york,your
0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,1,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0
1,1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,3,0,1,1,0,1,1,0,0,1,1,0,0,0,0,1,1,3,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,4,0,0,1,1,0,1,0,1,1,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0
4,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0
6,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0
7,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0
8,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0
9,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0


In [9]:
# print out 10 most common words in our data
df.sum().sort_values()[-10:]

recent      5
articles    5
by          5
york        6
times       6
new         6
cohen       7
of          8
and         8
the         9
dtype: int64

In [10]:
# print out 10 least common words in our data
df.sum().sort_values()[:10] # small sample size means most words occur one time

11           1
org          1
once         1
now          1
noam         1
mistrials    1
made         1
lively       1
our          1
lines        1
dtype: int64

# TF-IDF Conversion
We have a very small sample of data, but lets covert to tf-idf for the sake of programming it. Recall that Tf-idf transformation (default in `sklearn` is):

$$ \text{tf}(t,d) = f_{td}\text{, } t\in T \text{ and } d \in D $$

$$ \text{idf}(t,d) = \log{\frac{|D|}{|n_t|}}\text{, where } n_t=d\in D \text{ with } t\in d $$

$$\text{tf-idf}(t,d)=\text{tf}(t,d) \cdot (1+\text{idf}(t,d))$$

- http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer() # an object capable of counting words in a document!

tfidf_mat = tfidf_vect.fit_transform(summary_text) # that's it! its converted!!

In [10]:
# convert to pandas to get better idea about the data
df = pd.DataFrame(data=tfidf_mat.toarray(),columns=tfidf_vect.get_feature_names())
df

,2008,after,ailing,alternative,always,american,an,and,angry,are,arizona,as,at,avoided,backstage,barack,becomes,biggest,bill,bipartisanship,bush,carl,century,ceremony,chief,complex,correspondent,country,courage,critic,daily,decades,died,discussing,during,editors,eight,figure,for,formative,former,from,funny,george,had,half,have,he,here,hero,his,historic,honor,hulse,immaturity,in,is,it,larger,leaders,legacy,lesser,life,look,looks,losing,lost,loved,mccain,meant,mention,mentioning,military,mr,name,named,new,nominated,nomination,nominee,obama,of,offer,offered,often,on,or,palin,party,patriotism,place,praise,presence,president,presidential,public,race,reader,readers,reading,recalled,reflections,refusal,reporters,republican,sarah,saturday,says,senate,senator,served,shaped,shows,signing,sometimes,speech,spending,than,that,the,their,threat,through,times,to,today,trump,turmoil,vice,view,vindictiveness,war,washington,which,while,who,with,without,won,would,years,york
0,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.229322,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.30834,0.000000,0.000000,0.30834,0.000000,0.000000,0.000000,0.000000,0.30834,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.30834,0.000000,0.000000,0.000000,0.000000,0.183099,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.30834,0.000000,0.229322,0.000000,0.30834,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.00000,0.203883,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.262117,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.183099,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.227999,0.000000,0.000000,0.30834,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000
1,0.00000,0.00000,0.199489,0.000000,0.000000,0.000000,0.000000,0.104172,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.199489,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.234668,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.139351,0.000000,0.234668,0.000000,0.234668,0.174530,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.234668,0.000000,0.199489,0.155169,0.199489,0.234668,0.000000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.174530,0.000000,0.000000,0.00000,0.234668,0.000000,0.000000,0.000000,0.000000,0.234668,0.000000,0.000000,0.00000,0.000000,0.234668,0.000000,0.139351,0.000000,0.000000,0.234668,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.173523,0.000000,0.000000,0.00000,0.00000,0.174530,0.000000,0.199489,0.000000,0.00000,0.000000,0.234668,0.000000,0.000000,0.000000,0.234668,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000
2,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.145567,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.278761,0.000000,0.000000,0.000000,0.00000,0.000000,0.278761,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.327919,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000

In [11]:
# print out 10 words with max tfidf, normalized by document occurrence
df.max().sort_values()[-10:]

complex    0.308340
century    0.308340
daily      0.308340
offer      0.327919
praise     0.327919
hero       0.327919
readers    0.327919
war        0.327919
the        0.386105
times      0.401740
dtype: float64

# Working with (a bit) more data
What if we do not have the memory to deal with dense matrix representatioan and we need to keep it sparse?


In [12]:
from sklearn.datasets import fetch_20newsgroups
bunch = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)

the `bunch` object returned from sklearn is similar to a python dictionary. We can access different fields of the object with keys.

In [15]:
len(bunch.data)

11314

In [16]:
print(bunch.data[0]) # we should split this up by newlines

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [19]:
bunch.target_names[bunch.target[0]]

'rec.autos'

In [20]:
import numpy as np
#randomly grad an index
idx = round(np.random.rand()*len(bunch.data))
# and print it pretty
print("\n".join(bunch.data[idx].split("\n")))

From: kevin@rotag.mi.org (Kevin Darcy)
Subject: Re: Who be Conservative on this.....
Organization: Who, me???
Lines: 31

In article <1993Apr5.005204.29158@mnemosyne.cs.du.edu> mcochran@nyx.cs.du.edu (Mark A. Cochran) writes:
>kevin@rotag.mi.org (Kevin Darcy) writes:
>>In article <1993Apr2.155820.16998@mnemosyne.cs.du.edu> mcochran@nyx.cs.du.edu (Mark A. Cochran) writes:
>>>
>>>The SC allows restrictions after 'viability' (a term never medically defined) 
>>                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
>>
>>Any physician who can't make a reasonable stab at determining whether a given
>>fetus is viable or not is not qualified to perform an abortion. 
>
>Kebbin shows that he does not know the difference between determining
>the viability of an *individual* fetus, and providing the "universally
>accepted medical definition[s] of viability" 

I was not discussing "universal" definitions in this post, Mark. Please
refrain from dragging in irrelevancies.

Do yo

In [21]:
%%time
news_tfidf = tfidf_vect.fit_transform(bunch.data) 

CPU times: user 3.47 s, sys: 146 ms, total: 3.62 s
Wall time: 3.63 s


In [22]:
news_tfidf.shape

(11314, 130107)

In [23]:
tfidf_vect.vocabulary_

{'from': 56979,
 'lerxst': 75358,
 'wam': 123162,
 'umd': 118280,
 'edu': 50527,
 'where': 124031,
 'my': 85354,
 'thing': 114688,
 'subject': 111322,
 'what': 123984,
 'car': 37780,
 'is': 68532,
 'this': 114731,
 'nntp': 87620,
 'posting': 95162,
 'host': 64095,
 'rac3': 98949,
 'organization': 90379,
 'university': 118983,
 'of': 89362,
 'maryland': 79666,
 'college': 40998,
 'park': 92081,
 'lines': 76032,
 '15': 4605,
 'was': 123292,
 'wondering': 124931,
 'if': 65798,
 'anyone': 28615,
 'out': 90774,
 'there': 114579,
 'could': 42876,
 'enlighten': 51793,
 'me': 80638,
 'on': 89860,
 'saw': 104813,
 'the': 114455,
 'other': 90686,
 'day': 45295,
 'it': 68766,
 'door': 48618,
 'sports': 109581,
 'looked': 76718,
 'to': 115475,
 'be': 32311,
 'late': 74693,
 '60s': 16574,
 'early': 50111,
 '70s': 18299,
 'called': 37433,
 'bricklin': 34995,
 'doors': 48620,
 'were': 123796,
 'really': 99822,
 'small': 108252,
 'in': 66608,
 'addition': 26073,
 'front': 56989,
 'bumper': 35612,
 'se

In [24]:
# create pandas dataframe
vec = news_tfidf.max(axis=0)
df  = pd.DataFrame(data=vec.toarray(),columns=tfidf_vect.get_feature_names())

In [25]:
# largest tfidf 
df.max().sort_values()[-10:]

kk          0.870294
db          0.871473
scsi        0.875086
blah        0.879426
donoghue    0.891653
00          0.907726
___         0.908826
25          0.913127
forged      0.940511
ax          0.998314
dtype: float64

In [26]:
# now lets do the transformation with a smaller vocabulary
tfidf_vect = TfidfVectorizer(stop_words='english',
                             max_df=0.01,
                             min_df=4)
news_tfidf = tfidf_vect.fit_transform(bunch.data) 
print(news_tfidf.shape)
vec=news_tfidf.max(axis=0)
df = pd.DataFrame(data=vec.toarray(),columns=tfidf_vect.get_feature_names())
df.max().sort_values()[-10:]

(11314, 28592)


dialix            0.947270
blah              0.952881
ualberta          0.956563
stephanopoulos    0.959855
forged            0.971300
mufti             0.976947
ax                0.999881
meyers            1.000000
slower            1.000000
ucsd              1.000000
dtype: float64

# Using your own vocabulary

In [27]:
# read in scrabble dictionary from file
with open('data/ospd.txt') as f:
    vocab = f.read().split('\n')
    
# now lets do the transformation with a custom vocabulary
tfidf_vect = TfidfVectorizer(vocabulary=vocab)
news_tfidf = tfidf_vect.fit_transform(bunch.data) 
print(news_tfidf.shape)
vec=news_tfidf.max(axis=0)
df = pd.DataFrame(data=vec.toarray(),columns=tfidf_vect.get_feature_names())
df.max().sort_values()[-10:]

(11314, 79340)


incoming    0.925305
siemens     0.927114
water       0.928029
echo        0.947289
blah        0.951962
dos         0.953675
lib         0.954914
forged      0.978762
la          0.982737
ax          0.999999
dtype: float64

Looking for how to do a word cloud? Check this out:
- https://github.com/amueller/word_cloud

Want to perform more serious NLP with richer options:
- http://www.nltk.org